Import mario modules

In [1]:
#! /usr/bin/python3
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import *

Import plotting modules

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import random, sys

Setup plotting

In [4]:
%matplotlib inline
%matplotlib notebook
plt.ioff()

Setup enviroment

In [5]:
env = gym_super_mario_bros.make('SuperMarioBros-1-2-v0')
env = JoypadSpace(env, RIGHT_ONLY)

Setup parameters

In [6]:
#Animation
save_animation = 1 # Saves animation to plot
frames_skipped = 3 # How many frames to skip when creating animation

#Generation
generation_amount = 1000

#Population
population_amount = 30 # Amount of agents per generation

#Agent
moves_amount = 5000 # Max amount of moves agent can perform
moves_to_check = 100 # How many moves without increase in fitness until termination
mutation_rate = 1.001 # Chance of any move changing to a random move
moves_mutable = 0.2 # How many percent of moves are mutable (starting from end)
                    # Regardless of percent, a minimum of (moves_to_check*2) moves
                    # are mutable.


In [7]:
t = 0
def print_info(info, reward):
    global t 
    t += 1
    if not t % 100:
        print(info, reward)

In [8]:
def handle_frame(step, images):
    if not step % frames_skipped:
        image = plt.imshow(env.render(mode='rgb_array'))
        images.append([image])

In [9]:
fig = plt.figure()
def display_animation(images):
    anim = animation.ArtistAnimation(fig, images,
                                     interval=15*frames_skipped, blit=True)
    rc('animation', html='jshtml')
    return(anim)

In [10]:
class Agent:
    def __init__(self, fittest):
        self.fitness = 0
        self.images = []
        if fittest:
            self.moves = fittest.moves.copy()
            self.moves_used = fittest.moves_used
        else:
            self.moves = []
            self.moves_used = 0

In [11]:
def check_fitness(player, fitness, past_fitness):
    if player.fitness < fitness:
        player.fitness = fitness
    if len(past_fitness) < moves_to_check:
        past_fitness.append(fitness)
    else:
        past_fitness.pop(0)
        past_fitness.append(fitness)
        for i in range(moves_to_check):
            if past_fitness[i] > past_fitness[0]:
                break
            #Kills player if no progress in fitness for moves_to_check moves
            if i == moves_to_check - 1:
                return True

In [12]:
def mutate_moves(player):
    start_index = int(player.moves_used * (1-moves_mutable))
    if player.moves_used - start_index < moves_to_check * 2:
        start_index = player.moves_used - moves_to_check * 2
    for i in range(start_index, moves_amount):
        num = random.random()
        if num < mutation_rate:
            player.moves[i] = env.action_space.sample()

def agent_get_moves(player, population):
    if not player.moves:
        for i in range(moves_amount):
            player.moves.append(env.action_space.sample())
    else:
        if population:
            mutate_moves(player)

In [13]:
def print_mutation_info(fittest, population):
    global mutation_rate
    current_fittest = get_fittest(population)
    if fittest and fittest.fitness == current_fittest.fitness:
        print("Increasing mutation chance by 0.005")
        mutation_rate += 0.005
        print("Mutation chance is now {}%".format(mutation_rate*100))
    else:
        print("Fitness increased succesfully, restore mutation chance to 0.01")
        mutation_rate = 0.01
    int(mutable_amount) = current_fittest.moves_used * moves_mutable
    if mutable_amount < moves_to_check * 2:
        mutable_amount = moves_to_check * 2
    print("Mutating last {}% of moves, equal to {} moves".format(moves_mutable*100,
                                                                 mutable_amount))

In [14]:
def display_recording(player):
    print("Displaying player with fitness score {}.".format(
                                    player.fitness))
    display_animation(player)

In [15]:
def get_fittest(population):
    player_num = 0
    for i in range(1, len(population)):
        if population[i].fitness > population[player_num].fitness:
            player_num = i
    return population[player_num]

def record_player(player):
    images = []
    done = False
    observation = env.reset()
    past_fitness = []
    for move in range(len(player.moves)):
        if done:
            break
        state, reward, done, info = env.step(player.moves[move])
        if done:
            break
        done = check_fitness(player, info['x_pos'], past_fitness)
        handle_frame(move, images)
    return images
        
def play_generation(fittest, population):
    for i in range(population_amount):
        player = Agent(fittest)
        observation = env.reset()
        done = False
        past_fitness = []
        agent_get_moves(player, population)
        for move in range(len(player.moves)):
            if done:
                break
            state, reward, done, info = env.step(player.moves[move])
            if done:
                break
            done = check_fitness(player, info['x_pos'], past_fitness)
            env.render()
        player.moves_used = move
        population.append(player)
        print("Player {} achieved a fitness of {} in {} moves".format(i,
                                                               player.fitness, move))
        if info['flag_get']:
            return 1
    print_mutation_info(fittest, population)
    return 0

In [16]:
def custom_starting_agent(filename, fitness):
    fittest = Agent(None)
    fittest.fitness = fitness
    fittest.moves = open(filename, "r").read()
    fittest.moves = fittest.moves.strip("[]")
    fittest.moves = [int(s) for s in fittest.moves.split(',')]
    fittest.moves_used = 1120
    return (fittest)

fittest = None
#fittest = custom_starting_agent("2227-fitness", 900)
for generation in range(generation_amount):
    population = []
    if play_generation(fittest, population):
        break
    fittest = get_fittest(population)
    print("GENERATION {} HIGHEST FITNESS ACHIEVED: {}".format(generation,
                                                             fittest.fitness))

Player 0 achieved a fitness of 184 in 89 moves


/home/sadawi/.local/lib/python3.6/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Player 1 achieved a fitness of 482 in 340 moves
Player 2 achieved a fitness of 183 in 93 moves
Player 3 achieved a fitness of 459 in 243 moves
Player 4 achieved a fitness of 177 in 93 moves
Player 5 achieved a fitness of 354 in 442 moves
Player 6 achieved a fitness of 291 in 245 moves
Player 7 achieved a fitness of 200 in 87 moves
Player 8 achieved a fitness of 290 in 214 moves
Player 9 achieved a fitness of 482 in 325 moves
Player 10 achieved a fitness of 670 in 389 moves
Player 11 achieved a fitness of 322 in 245 moves
Player 12 achieved a fitness of 906 in 493 moves
Player 13 achieved a fitness of 355 in 336 moves
Player 14 achieved a fitness of 322 in 355 moves
Player 15 achieved a fitness of 323 in 262 moves
Player 16 achieved a fitness of 179 in 95 moves
Player 17 achieved a fitness of 483 in 435 moves
Player 18 achieved a fitness of 484 in 326 moves
Player 19 achieved a fitness of 484 in 281 moves
Player 20 achieved a fitness of 388 in 305 moves
Player 21 achieved a fitness of 1

Player 4 achieved a fitness of 1180 in 615 moves
Player 5 achieved a fitness of 1326 in 657 moves
Player 6 achieved a fitness of 1326 in 657 moves
Player 7 achieved a fitness of 979 in 529 moves
Player 8 achieved a fitness of 1326 in 657 moves
Player 9 achieved a fitness of 1326 in 657 moves
Player 10 achieved a fitness of 1326 in 657 moves
Player 11 achieved a fitness of 1326 in 657 moves
Player 12 achieved a fitness of 1320 in 692 moves
Player 13 achieved a fitness of 1326 in 657 moves
Player 14 achieved a fitness of 1326 in 657 moves
Player 15 achieved a fitness of 1326 in 657 moves
Player 16 achieved a fitness of 1325 in 657 moves
Player 17 achieved a fitness of 1326 in 657 moves
Player 18 achieved a fitness of 1326 in 657 moves
Player 19 achieved a fitness of 1326 in 657 moves
Player 20 achieved a fitness of 1326 in 657 moves
Player 21 achieved a fitness of 1326 in 657 moves
Player 22 achieved a fitness of 1326 in 657 moves
Player 23 achieved a fitness of 1325 in 657 moves
Player 

Player 4 achieved a fitness of 1636 in 900 moves
Player 5 achieved a fitness of 1636 in 896 moves
Player 6 achieved a fitness of 1635 in 896 moves
Player 7 achieved a fitness of 1636 in 896 moves
Player 8 achieved a fitness of 1636 in 896 moves
Player 9 achieved a fitness of 1636 in 896 moves
Player 10 achieved a fitness of 1636 in 896 moves
Player 11 achieved a fitness of 1636 in 896 moves
Player 12 achieved a fitness of 1636 in 896 moves
Player 13 achieved a fitness of 1636 in 896 moves
Player 14 achieved a fitness of 1636 in 896 moves
Player 15 achieved a fitness of 1635 in 896 moves
Player 16 achieved a fitness of 1636 in 896 moves
Player 17 achieved a fitness of 1636 in 896 moves
Player 18 achieved a fitness of 1636 in 896 moves
Player 19 achieved a fitness of 1636 in 896 moves
Player 20 achieved a fitness of 1554 in 757 moves
Player 21 achieved a fitness of 1636 in 896 moves
Player 22 achieved a fitness of 1636 in 896 moves
Player 23 achieved a fitness of 1636 in 896 moves
Player

Player 2 achieved a fitness of 1731 in 1031 moves
Player 3 achieved a fitness of 1731 in 1031 moves
Player 4 achieved a fitness of 1731 in 1031 moves
Player 5 achieved a fitness of 1731 in 1031 moves
Player 6 achieved a fitness of 1730 in 1029 moves
Player 7 achieved a fitness of 1731 in 1031 moves
Player 8 achieved a fitness of 1731 in 1027 moves
Player 9 achieved a fitness of 1731 in 1031 moves
Player 10 achieved a fitness of 1731 in 1027 moves
Player 11 achieved a fitness of 1731 in 1031 moves
Player 12 achieved a fitness of 1731 in 1031 moves
Player 13 achieved a fitness of 1731 in 1031 moves
Player 14 achieved a fitness of 1730 in 1030 moves
Player 15 achieved a fitness of 1731 in 1031 moves
Player 16 achieved a fitness of 1731 in 1031 moves
Player 17 achieved a fitness of 1731 in 1031 moves
Player 18 achieved a fitness of 1731 in 1027 moves
Player 19 achieved a fitness of 1636 in 896 moves
Player 20 achieved a fitness of 1731 in 1031 moves
Player 21 achieved a fitness of 1731 in 

Player 0 achieved a fitness of 1732 in 1017 moves
Player 1 achieved a fitness of 1732 in 1017 moves
Player 2 achieved a fitness of 1732 in 1017 moves
Player 3 achieved a fitness of 1732 in 1017 moves
Player 4 achieved a fitness of 1732 in 1027 moves
Player 5 achieved a fitness of 1732 in 1017 moves
Player 6 achieved a fitness of 1731 in 1017 moves
Player 7 achieved a fitness of 1732 in 1017 moves
Player 8 achieved a fitness of 1732 in 1017 moves
Player 9 achieved a fitness of 1732 in 1017 moves
Player 10 achieved a fitness of 1730 in 1028 moves
Player 11 achieved a fitness of 1732 in 1017 moves
Player 12 achieved a fitness of 1732 in 1017 moves
Player 13 achieved a fitness of 1732 in 1017 moves
Player 14 achieved a fitness of 1732 in 1017 moves
Player 15 achieved a fitness of 1732 in 1017 moves
Player 16 achieved a fitness of 1732 in 1017 moves
Player 17 achieved a fitness of 1730 in 1016 moves
Player 18 achieved a fitness of 1730 in 1028 moves
Player 19 achieved a fitness of 1732 in 1

Player 0 achieved a fitness of 1732 in 1017 moves
Player 1 achieved a fitness of 1732 in 1017 moves
Player 2 achieved a fitness of 1732 in 1017 moves
Player 3 achieved a fitness of 1732 in 1017 moves
Player 4 achieved a fitness of 1732 in 1017 moves
Player 5 achieved a fitness of 1730 in 1027 moves
Player 6 achieved a fitness of 1636 in 896 moves
Player 7 achieved a fitness of 1636 in 896 moves
Player 8 achieved a fitness of 1732 in 1017 moves
Player 9 achieved a fitness of 1636 in 896 moves
Player 10 achieved a fitness of 1732 in 1017 moves
Player 11 achieved a fitness of 1730 in 1017 moves
Player 12 achieved a fitness of 1636 in 896 moves
Player 13 achieved a fitness of 1732 in 1017 moves
Player 14 achieved a fitness of 1732 in 1017 moves
Player 15 achieved a fitness of 1732 in 1017 moves
Player 16 achieved a fitness of 1732 in 1017 moves
Player 17 achieved a fitness of 1732 in 1017 moves
Player 18 achieved a fitness of 1730 in 1028 moves
Player 19 achieved a fitness of 1731 in 1017 

Player 0 achieved a fitness of 1732 in 1017 moves
Player 1 achieved a fitness of 1636 in 896 moves
Player 2 achieved a fitness of 1732 in 1024 moves
Player 3 achieved a fitness of 1636 in 896 moves
Player 4 achieved a fitness of 1636 in 896 moves
Player 5 achieved a fitness of 1636 in 896 moves


KeyboardInterrupt: 

Animate results

In [ ]:
winner = get_fittest(population)
print("Player from generation {} won! Achieving a fitness of {} in {} moves!"
          .format(generation, winner.fitness, winner.moves_used))
print("Compiling animation...")
ani = display_animation(record_player(winner))
ani

Save animation to file

In [ ]:
Writer = animation.FFMpegFileWriter
writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=-1)
ani.save("mario_flag_get-{}.mp4".format(random.random()), writer = writer)

In [ ]:
env.close()
plt.close('all')

In [17]:
print(SIMPLE_MOVEMENT)
print(env)
print(population[0].moves)

[['NOOP'], ['right'], ['right', 'A'], ['right', 'B'], ['right', 'A', 'B'], ['A'], ['left']]
<JoypadSpace<TimeLimit<SuperMarioBrosEnv<SuperMarioBros-1-2-v0>>>>
[1, 0, 4, 4, 0, 4, 1, 2, 4, 0, 3, 2, 1, 1, 4, 1, 0, 4, 2, 2, 1, 1, 2, 3, 2, 4, 4, 1, 1, 3, 2, 0, 0, 4, 2, 3, 4, 1, 3, 2, 0, 4, 0, 4, 4, 0, 1, 3, 3, 2, 0, 1, 1, 3, 4, 3, 0, 0, 0, 2, 3, 2, 4, 2, 2, 2, 3, 2, 3, 3, 1, 1, 2, 2, 4, 4, 2, 1, 4, 3, 3, 1, 1, 2, 1, 4, 0, 3, 0, 4, 4, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 2, 4, 1, 4, 3, 3, 1, 0, 2, 2, 4, 3, 1, 3, 0, 4, 4, 0, 0, 4, 2, 1, 3, 0, 4, 1, 2, 2, 2, 2, 4, 1, 1, 1, 2, 0, 1, 4, 3, 4, 3, 3, 1, 4, 4, 0, 2, 4, 3, 1, 2, 4, 0, 2, 4, 2, 1, 3, 4, 1, 0, 3, 4, 3, 1, 0, 0, 1, 4, 0, 3, 0, 3, 4, 2, 4, 2, 2, 3, 1, 0, 2, 4, 4, 2, 3, 2, 2, 1, 4, 4, 0, 4, 2, 1, 2, 0, 1, 0, 1, 2, 0, 3, 3, 2, 4, 1, 4, 1, 3, 3, 1, 3, 0, 3, 4, 0, 1, 2, 1, 3, 2, 2, 4, 2, 3, 4, 4, 1, 0, 0, 4, 2, 3, 3, 4, 1, 3, 2, 0, 3, 4, 2, 0, 4, 2, 4, 3, 3, 3, 2, 0, 3, 0, 3, 0, 2, 3, 2, 0, 1, 0, 0, 2, 1, 3, 4, 0, 3, 1, 4, 4, 3, 0, 3, 0, 4, 4, 0, 